# Init

In [2]:
import os
os.chdir('..')
os.getcwd()

'/Users/wliao0504/code/clif/CLIF-MIMIC'

In [3]:
import numpy as np
import pandas as pd
import logging
import duckdb
import pandera as pa
from importlib import reload
import src.utils as utils
from typing import Dict, List
# reload(utils)
from src.utils import construct_mapper_dict, fetch_mimic_events, load_mapping_csv, \
    get_relevant_item_ids, find_duplicates, rename_and_reorder_cols, save_to_rclif, \
    convert_and_sort_datetime, setup_logging, con, REPO_ROOT, mimic_table_pathfinder, \
    search_mimic_items, read_from_rclif
from src.utils_qa import all_null_check

loaded configuration from /Users/wliao0504/code/clif/CLIF-MIMIC/src/../config/config.json


# MIMIC item query

In [21]:
mc_mcide = pd.read_csv("https://raw.githubusercontent.com/Common-Longitudinal-ICU-data-Format/CLIF/refs/heads/main/mCIDE/microbiology_culture/clif_microbiology_culture_organism_categories.csv")

q = f"""
SELECT organism_category
    , split(organism_category, '_')[1] as genus
    , split(organism_category, '_')[2] as species
FROM mc_mcide
"""
df = duckdb.sql(q).df()
df

,organism_category,genus,species
0,abiotrophia_defectiva,abiotrophia,defectiva
1,achromobacter_sp,achromobacter,sp
2,achromobacter_xylosoxidans,achromobacter,xylosoxidans
3,acidaminococcus_sp,acidaminococcus,sp
4,acinetobacter_baumanii,acinetobacter,baumanii
...,...,...,...
528,weisella_confusa,weisella,confusa
529,weissella_confusa,weissella,confusa
530,xanthomonas_sp,xanthomonas,sp
531,yeast (nos),yeast (nos),None


In [12]:
mc_mcide['organism_category'].str.split("_")

0           [abiotrophia, defectiva]
1                [achromobacter, sp]
2      [achromobacter, xylosoxidans]
3              [acidaminococcus, sp]
4          [acinetobacter, baumanii]
                   ...              
528              [weisella, confusa]
529             [weissella, confusa]
530                [xanthomonas, sp]
531                    [yeast (nos)]
532                 [zygomycete, sp]
Name: organism_category, Length: 533, dtype: object

In [ ]:
# utils.resave_mimic_table_from_csv_to_parquet("microbiologyevents")

q = f"""
SELECT *
FROM '{mimic_table_pathfinder("microbiologyevents")}'
LIMIT 10
"""
df = duckdb.sql(q).df()
df

,microevent_id,subject_id,hadm_id,micro_specimen_id,order_provider_id,chartdate,charttime,spec_itemid,spec_type_desc,test_seq,...,org_name,isolate_num,quantity,ab_itemid,ab_name,dilution_text,dilution_comparison,dilution_value,interpretation,comments
0,1,10000032,<NA>,1304715,P69FQC,2180-03-23,2180-03-23 11:51:00,70046,IMMUNOLOGY,1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,___
1,2,10000032,<NA>,3342526,P69FQC,2180-03-23,2180-03-23 11:51:00,70093,Blood (Toxo),1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,NEGATIVE FOR TOXOPLASMA IgG ANTIBODY BY EIA. ...
2,3,10000032,<NA>,3910370,P69FQC,2180-03-23,2180-03-23 11:51:00,70017,SEROLOGY/BLOOD,1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,NONREACTIVE. Reference Range: Non-Reactive.
3,4,10000032,<NA>,5401234,P69FQC,2180-03-23,2180-03-23 11:51:00,70017,SEROLOGY/BLOOD,1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,POSITIVE BY EIA. A positive IgG result genera...
4,5,10000032,<NA>,6287540,P69FQC,2180-03-23,2180-03-23 11:51:00,70046,IMMUNOLOGY,1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,HIV-1 RNA is not detected. Performed using th...
5,6,10000032,<NA>,7322675,P69FQC,2180-03-23,2180-03-23 11:51:00,70017,SEROLOGY/BLOOD,1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,POSITIVE BY EIA. A positive IgG result genera...
6,7,10000032,<NA>,8934175,P69FQC,2180-03-23,2180-03-23 11:51:00,70017,SEROLOGY/BLOOD,1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,POSITIVE BY EIA. A positive IgG result genera...
7,8,10000032,<NA>,9041996,P69FQC,2180-03-23,2180-03-23 11:51:00,70088,Blood (EBV),1,...,None,<NA>,None,<NA>,None,None,None,NaN,None,POSITIVE BY EIA.
8,9,10000032,<NA>,9041996,P69FQC,2180-03-23,2180-03-23 11:51:00,70088,Blood (EBV),2,...,None,<NA>,None,<NA>,None,None,None,NaN,None,POSITIVE BY EIA.
9,10,10000032,<NA>,9041996,P69FQC,2180-03-23,2180-03-23 11:51:00,70088,Blood (EBV),3,...,None,<NA>,None,<NA>,None,None,None,NaN,None,NEGATIVE <1:10 BY IFA. INTERPRETATION: RESULT...
